## Setup

### Import libraries

In [1]:
from benchmarks.experiments.wrapper_overhead.common.deployment_utility import WrapperOverheadDeploymentUtility

### Setup Common Resources

In [2]:
region = 'us-east-2'
deployment_utility: WrapperOverheadDeploymentUtility = WrapperOverheadDeploymentUtility(region)

## Deploy Associated AWS functions

### DNA Visualization

#### Boto3 Direct Calls

In [22]:
dna_vis_direct_calls_dir = "./dna_visualization/external_database/boto3_only_direct_calls" # Boto3 only direct calls
deployment_utility.deploy_experiment(dna_vis_direct_calls_dir)

Deploying boto3_direct workload
workflow-wo-dna_vis-ed-direct

Deploying wo-dna_vis-ed-direct-visualize
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-dna_vis-ed-direct-visualize-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.2s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:9ae4af1397c3c7ac09aca53cca2621c9fdd5f0a2674f2bf05920888d24883f91
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 108.82kB done
#5 DONE 0.0s

#6 [2/6] COPY requirements.txt ./
#6 CACHED

#7 [4/6] COPY app.py ./
#7 CACHED

#8 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#8 CACHED

#9 [5/6] COPY src ./src
#9 CACHED

#10 [6/6] COPY multi_x_serverless ./multi_x_serverless
#10 CACHED

#11 exporting to image
#11 exporting layers done
#11 writing image sha256:05eeb6d255b2f613b77cd26af60ae066fabea0d11f5b7994162bea5e0e880c8b done
#11 naming to docker.io/library/wo-dna_vis-ed-direct-visualize:

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct-visualize]
2b945486522c: Preparing
3d69f38a8b7a: Preparing
a487354e1091: Preparing
8e1b406f394f: Preparing
3a89be36bfcd: Preparing
0a20c8da2344: Preparing
05788b7af5ed: Preparing
706e59713fa4: Preparing
052242efe275: Preparing
274a79fec6a9: Preparing
6849347e354b: Preparing
05788b7af5ed: Waiting
706e59713fa4: Waiting
052242efe275: Waiting
274a79fec6a9: Waiting
6849347e354b: Waiting
0a20c8da2344: Waiting
3d69f38a8b7a: Layer already exists
3a89be36bfcd: Layer already exists
8e1b406f394f: Layer already exists
a487354e1091: Layer already exists
2b945486522c: Layer already exists
0a20c8da2344: Layer already exists
05788b7af5ed: Layer already exists
274a79fec6a9: Layer already exists
706e59713fa4: Layer already exists
052242efe275: Layer already exists
6849347e354b: Layer already exists


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct-visualize:latest to ECR.


latest: digest: sha256:1e3287d058c482c86b35019c11af0e7014363bb2a4d9753757b08b4ae74c4c37 size: 2624
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-dna_vis-ed-direct-visualize

Deploying wo-dna_vis-ed-direct-get_input
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-dna_vis-ed-direct-get_input-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.1s

#3 [internal] load .dockerignore
#3 transferring context:
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:9ae4af1397c3c7ac09aca53cca2621c9fdd5f0a2674f2bf05920888d24883f91
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 106.69kB done
#5 DONE 0.0s

#6 [5/6] COPY src ./src
#6 CACHED

#7 [2/6] COPY requirements.txt ./
#7 CACHED

#8 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#8 CACHED

#9 [4/6] COPY app.py ./
#9 CACHED

#10 [6/6] COPY multi_x_serverless ./multi_x_serverless
#10 CACHED

#11 exporting to image
#11 exporting layers done
#11 writing image sha256:a47657b0c433b9c7fe387c8f3e7a3c7b9ba2c5e83b8c42801c455253d8bfc198 done
#11 naming to docker.io/library/wo-dna

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct-get_input]
882ab2a529f2: Preparing
49721b2ff3a4: Preparing
9c922acf8575: Preparing
6fe8485efa46: Preparing
34352bc5dfd0: Preparing
0a20c8da2344: Preparing
05788b7af5ed: Preparing
706e59713fa4: Preparing
052242efe275: Preparing
274a79fec6a9: Preparing
6849347e354b: Preparing
274a79fec6a9: Waiting
706e59713fa4: Waiting
052242efe275: Waiting
6849347e354b: Waiting
0a20c8da2344: Waiting
05788b7af5ed: Waiting
9c922acf8575: Layer already exists
882ab2a529f2: Layer already exists
34352bc5dfd0: Layer already exists
49721b2ff3a4: Layer already exists
6fe8485efa46: Layer already exists
0a20c8da2344: Layer already exists
274a79fec6a9: Layer already exists
706e59713fa4: Layer already exists
052242efe275: Layer already exists
05788b7af5ed: Layer already exists
6849347e354b: Layer already exists


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct-get_input:latest to ECR.


latest: digest: sha256:a1a45d13a9a038f76f70d7ea281e4a1e64e881f153e021759ba601c4adedb60f size: 2623
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-dna_vis-ed-direct-get_input
Completed deployment of workflow-wo-dna_vis-ed-direct




True

#### Boto3 SNS calls

In [23]:
dna_vis_sns_calls_dir = "./dna_visualization/external_database/boto3_only_sns" # Boto3 only sns calls
deployment_utility.deploy_experiment(dna_vis_sns_calls_dir)

Deploying boto3_sns workload
workflow-wo-dna_vis-ed-direct_sns

Deploying wo-dna_vis-ed-direct_sns-visualize
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-dna_vis-ed-direct_sns-visualize-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.2s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:9ae4af1397c3c7ac09aca53cca2621c9fdd5f0a2674f2bf05920888d24883f91
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 108.89kB done
#5 DONE 0.0s

#6 [2/6] COPY requirements.txt ./
#6 CACHED

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 CACHED

#8 [4/6] COPY app.py ./
#8 DONE 0.0s

#9 [5/6] COPY src ./src
#9 DONE 0.0s

#10 [6/6] COPY multi_x_serverless ./multi_x_serverless
#10 DONE 0.0s

#11 exporting to image
#11 exporting layers 0.0s done
#11 writing image sha256:1ce368045b5cc215ce57ce037eae30a97b9bacbf6d1b4118b53a5e77c8cfd6ea done
#11 naming to docker.io/library/wo-dna_vis-ed-dir

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct_sns-visualize]
79cde3b2ad05: Preparing
119481f27e36: Preparing
ae411482cccc: Preparing
8e1b406f394f: Preparing
3a89be36bfcd: Preparing
0a20c8da2344: Preparing
05788b7af5ed: Preparing
706e59713fa4: Preparing
052242efe275: Preparing
274a79fec6a9: Preparing
6849347e354b: Preparing
706e59713fa4: Waiting
0a20c8da2344: Waiting
05788b7af5ed: Waiting
274a79fec6a9: Waiting
6849347e354b: Waiting
052242efe275: Waiting
8e1b406f394f: Layer already exists
3a89be36bfcd: Layer already exists
05788b7af5ed: Layer already exists
0a20c8da2344: Layer already exists
706e59713fa4: Layer already exists
052242efe275: Layer already exists
274a79fec6a9: Layer already exists
6849347e354b: Layer already exists
ae411482cccc: Pushed
119481f27e36: Pushed
79cde3b2ad05: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct_sns-visualize:latest to ECR.


latest: digest: sha256:b6dc50ee8a0124298f5883b7dbb0c53b72591a6c3ce0c1e0c7ffc91ad812f53a size: 2624
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-dna_vis-ed-direct_sns-visualize
Creating sns topic and subscribing lambda function
Resulting Topic ARN: arn:aws:sns:us-east-2:485595969306:wo-dna_vis-ed-direct_sns-visualize-sns_topic
Resulting Subscription ARN: arn:aws:sns:us-east-2:485595969306:wo-dna_vis-ed-direct_sns-visualize-sns_topic:8f6a0835-cf76-4577-9c33-288cb39b5a5c

Deploying wo-dna_vis-ed-direct_sns-get_input
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-dna_vis-ed-direct_sns-get_input-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.1s

#3 [internal] load .dockerignore
#3 transferring context:
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:9ae4af1397c3c7ac09aca53cca2621c9fdd5f0a2674f2bf05920888d24883f91
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 107.14kB done
#5 DONE 0.0s

#6 [2/6] COPY requirements.txt ./
#6 CACHED

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 CACHED

#8 [4/6] COPY app.py ./
#8 DONE 0.0s

#9 [5/6] COPY src ./src
#9 DONE 0.0s

#10 [6/6] COPY multi_x_serverless ./multi_x_serverless
#10 DONE 0.0s

#11 exporting to image
#11 exporting layers 0.0s done
#11 writing image sha256:35df3fdc8e91980b3b28f578c10307496214e394be81e25d200ce332001fcc1c done
#11 naming to docker.io/

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct_sns-get_input]
78ed9b80978c: Preparing
4f97db2c5e2b: Preparing
5422b33b9934: Preparing
6fe8485efa46: Preparing
34352bc5dfd0: Preparing
0a20c8da2344: Preparing
05788b7af5ed: Preparing
706e59713fa4: Preparing
052242efe275: Preparing
274a79fec6a9: Preparing
6849347e354b: Preparing
05788b7af5ed: Waiting
274a79fec6a9: Waiting
6849347e354b: Waiting
706e59713fa4: Waiting
052242efe275: Waiting
0a20c8da2344: Waiting
34352bc5dfd0: Layer already exists
6fe8485efa46: Layer already exists
0a20c8da2344: Layer already exists
05788b7af5ed: Layer already exists
706e59713fa4: Layer already exists
052242efe275: Layer already exists
274a79fec6a9: Layer already exists
6849347e354b: Layer already exists
4f97db2c5e2b: Pushed
5422b33b9934: Pushed
78ed9b80978c: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct_sns-get_input:latest to ECR.


latest: digest: sha256:4508bc80b6dfe6d795504f512de73013894ca9e76524646e2f57fc04f06cd529 size: 2623
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-dna_vis-ed-direct_sns-get_input
Creating sns topic and subscribing lambda function
Resulting Topic ARN: arn:aws:sns:us-east-2:485595969306:wo-dna_vis-ed-direct_sns-get_input-sns_topic
Resulting Subscription ARN: arn:aws:sns:us-east-2:485595969306:wo-dna_vis-ed-direct_sns-get_input-sns_topic:182ed0e2-8468-4893-9e6f-c788cf210a2c
Completed deployment of workflow-wo-dna_vis-ed-direct_sns




True

#### AWS Step Function

In [24]:
dna_vis_aws_step_functions_dir = "./dna_visualization/external_database/aws_step_function" # AWS Step Functions
deployment_utility.deploy_experiment(dna_vis_aws_step_functions_dir)

Deploying aws_step_function workload
wo-dna_vis-ed-step_function

Deploying wo-dna_vis-ed-step_function-visualize
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-dna_vis-ed-step_function-visualize-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.2s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:9ae4af1397c3c7ac09aca53cca2621c9fdd5f0a2674f2bf05920888d24883f91
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 108.89kB done
#5 DONE 0.0s

#6 [2/6] COPY requirements.txt ./
#6 CACHED

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 CACHED

#8 [4/6] COPY app.py ./
#8 DONE 0.0s

#9 [5/6] COPY src ./src
#9 DONE 0.0s

#10 [6/6] COPY multi_x_serverless ./multi_x_serverless
#10 DONE 0.0s

#11 exporting to image
#11 exporting layers 0.0s done
#11 writing image sha256:5cfa003e5b36adb2362095ca91d8275ab13197d8337f117f0ec11bfab6959b5c done
#11 naming to docker.io/library/wo-dna_vis-ed-ste

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-step_function-visualize]
9df28a6a8d0d: Preparing
a8f9a66cb326: Preparing
c1e1781d79d8: Preparing
8e1b406f394f: Preparing
3a89be36bfcd: Preparing
0a20c8da2344: Preparing
05788b7af5ed: Preparing
706e59713fa4: Preparing
052242efe275: Preparing
274a79fec6a9: Preparing
6849347e354b: Preparing
05788b7af5ed: Waiting
706e59713fa4: Waiting
6849347e354b: Waiting
052242efe275: Waiting
274a79fec6a9: Waiting
0a20c8da2344: Waiting
3a89be36bfcd: Layer already exists
8e1b406f394f: Layer already exists
0a20c8da2344: Layer already exists
05788b7af5ed: Layer already exists
706e59713fa4: Layer already exists
052242efe275: Layer already exists
6849347e354b: Layer already exists
274a79fec6a9: Layer already exists
a8f9a66cb326: Pushed
c1e1781d79d8: Pushed
9df28a6a8d0d: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-step_function-visualize:latest to ECR.


latest: digest: sha256:781efce4e440e1ecaac77033f6960c4cf6b747b6230d89ce49078a46a17766e4 size: 2624
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-dna_vis-ed-step_function-visualize

Deploying wo-dna_vis-ed-step_function-get_input
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-dna_vis-ed-step_function-get_input-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.1s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:9ae4af1397c3c7ac09aca53cca2621c9fdd5f0a2674f2bf05920888d24883f91
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 106.27kB done
#5 DONE 0.0s

#6 [2/6] COPY requirements.txt ./
#6 CACHED

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 CACHED

#8 [4/6] COPY app.py ./
#8 DONE 0.0s

#9 [5/6] COPY src ./src
#9 DONE 0.0s

#10 [6/6] COPY multi_x_serverless ./multi_x_serverless
#10 DONE 0.0s

#11 exporting to image
#11 exporting layers 0.0s done
#11 writing image sha256:fbbd28bd98bffefaee60116f5bb26663bf76da7c8e809a63dc0cf1b8d0654aed done
#11 naming to docker.io/library/wo-dna_vis-ed-ste

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-step_function-get_input]
edbf480faeb2: Preparing
e41be327bb84: Preparing
52c0086b9e60: Preparing
18f3008705b6: Preparing
1c58f3f28eec: Preparing
0a20c8da2344: Preparing
05788b7af5ed: Preparing
706e59713fa4: Preparing
052242efe275: Preparing
274a79fec6a9: Preparing
6849347e354b: Preparing
706e59713fa4: Waiting
052242efe275: Waiting
274a79fec6a9: Waiting
6849347e354b: Waiting
0a20c8da2344: Waiting
05788b7af5ed: Waiting
18f3008705b6: Layer already exists
1c58f3f28eec: Layer already exists
0a20c8da2344: Layer already exists
05788b7af5ed: Layer already exists
052242efe275: Layer already exists
706e59713fa4: Layer already exists
274a79fec6a9: Layer already exists
6849347e354b: Layer already exists
52c0086b9e60: Pushed
e41be327bb84: Pushed
edbf480faeb2: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-step_function-get_input:latest to ECR.


latest: digest: sha256:5cf9292cc0cbf3c6ba258fa8eb229c57739a4a167a9d61dc4ea9c6617d9b486e size: 2623
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-dna_vis-ed-step_function-get_input
Completed deployment of wo-dna_vis-ed-step_function


Delete role arn
Old resources for Removed
Creating state machine iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/state_machine-wo-dna_vis-ed-step_function-policy
Updating state machine
Resulting State Machine ARN: arn:aws:states:us-east-2:485595969306:stateMachine:state_machine-wo-dna_vis-ed-step_function
Completed deployment of wo-dna_vis-ed-step_function




True

#### Multi-X

- For MultiX, please set it up according to readme contained in the file. "./dna_visualization/external_database/multi_x"